# Theme

This project allows users to pick a real-world scenario (like phone launches, RBI policy changes, military AI adoption) and uses a Gemini-based AI agent to predict the probability of that event happening within a given time horizon, along with a reasoning-based forecast.

# Install

gradio → Creates the web interface to interact with the forecasting agent.

langchain → Provides tools to structure and manage prompts and responses for LLM workflows.

langchain-google-genai → Connects LangChain with Google’s Gemini model for event forecasting.

In [ ]:
!pip install -q gradio langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.


Imports Cell → Loads libraries for UI, environment handling, Gemini API access, and LangChain messaging.

In [ ]:
import os
import json
import gradio as gr
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage

Gemini API Key set up

Step-1: Go to Google AI Studio https://ai.google.dev/?utm_source=chatgpt.com

Step-2: Sign in with your Google account.

Step-3: Open the “API Keys” page. In the left sidebar, click on “API keys”.

Step-4: Create or select a project.

Step-5: Create a new API key.

Step-6: Copy and store the key safely.

API Setup

In [ ]:
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get("ENter Gemini API Key")   #Enter Gemini API key
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0.1)
except Exception as e:
    print("⚠️ No Gemini key found in Colab userdata as 'gem_Key'. LLM will be disabled.")
    llm = None

Scenarios Cell → Defines pre-built real-world business and policy events that users can select for forecasting.

Scenario Map Cell → Creates a dictionary for quick lookup of scenario details based on title.

In [ ]:
SCENARIOS = [
    {
        "id": "E1",
        "title": "Samsung Flagship Phone Launch",
        "context": (
            "Samsung is preparing to launch its next flagship Galaxy series smartphone in India.\n"
            "- The device is rumored to include advanced AI camera features and on-device AI processing.\n"
            "- Pre-launch marketing has started on YouTube, Instagram, and TV.\n"
            "- Competing brands like Xiaomi and OnePlus have recently increased prices slightly.\n"
            "- Last year's flagship saw strong pre-orders, especially in metro cities.\n"
        ),
        "event_to_forecast": (
            "Samsung's new flagship Galaxy phone will achieve record pre-orders in India "
            "compared to its previous flagship launch."
        ),
        "horizon": "Next 30 days"
    },
    {
        "id": "E2",
        "title": "Myntra Big Fashion Sale Performance",
        "context": (
            "Myntra has announced a Big Fashion Sale with aggressive discounts up to 70%.\n"
            "- Last festive season, Myntra reported its highest-ever daily orders during a similar sale.\n"
            "- Social media campaigns with influencers have started trending.\n"
            "- Competing platforms like Amazon Fashion and Ajio are also planning parallel offers.\n"
            "- Overall online fashion demand in India is growing steadily.\n"
        ),
        "event_to_forecast": (
            "Myntra will record a new highest single-day Gross Merchandise Value (GMV) "
            "during this sale period."
        ),
        "horizon": "During the upcoming sale (within 7 days)"
    },
    {
        "id": "E3",
        "title": "RBI Repo Rate Decision",
        "context": (
            "Inflation in India has eased compared to peak levels last year.\n"
            "- CPI inflation has been trending closer to the RBI's target range.\n"
            "- Global interest rate cycles are slowly shifting towards cuts in some major economies.\n"
            "- Domestic growth is stable but there are concerns about private investment slowdown.\n"
            "- RBI has maintained the repo rate for several consecutive policy meetings.\n"
        ),
        "event_to_forecast": (
            "The Reserve Bank of India (RBI) will reduce the repo rate in one of the next two "
            "monetary policy meetings."
        ),
        "horizon": "Next 6 months"
    },
    {
        "id": "E4",
        "title": "Indian Military AI Drone Adoption",
        "context": (
            "India is focusing on indigenous defence production and AI-based warfare technologies.\n"
            "- DRDO and private defence startups are actively testing AI-powered surveillance drones.\n"
            "- The government has announced multiple schemes to support defence tech startups.\n"
            "- Recent border security concerns have increased demand for real-time intelligence and monitoring.\n"
            "- Several trials of unmanned aerial vehicles (UAVs) have reportedly been successful.\n"
        ),
        "event_to_forecast": (
            "The Indian military will formally approve large-scale deployment of AI-powered "
            "surveillance drones for border monitoring."
        ),
        "horizon": "Within the next 12 months"
    },
    {
        "id": "E5",
        "title": "Custom (User-Defined Scenario)",
        "context": "",
        "event_to_forecast": "",
        "horizon": ""
    }
]

scenario_map = {s["title"]: s for s in SCENARIOS}


What this does:

Creates predefined topics so users can pick easily

Also supports Custom if the user wants their own case

forecast_event() Function Cell → Calls Gemini via LangChain to calculate event probability, label it, and return reasoning in JSON.

In [ ]:
def forecast_event(context: str, event_to_forecast: str, horizon: str):
    """
    Ask the LLM to give:
    - probability (0-100)
    - label (very unlikely / unlikely / uncertain / likely / very likely)
    - short reasoning
    """
    if llm is None:
        return {
            "probability": 50,
            "label": "unknown (no LLM configured)",
            "reasoning": "LLM is not configured – returning neutral default."
        }

    system_prompt = """
    You are an Event Forecasting AI Agent.
    You estimate how likely a future event is, based on the given context.
    Domain examples include: smartphones, Indian e-commerce, RBI policy, Indian defence adoption, etc.

    You MUST:
    - Be cautious (do not be over-confident).
    - Consider multiple factors: historical patterns, incentives, risks, and uncertainty.
    - Return ONLY valid JSON with this exact format:

    {
      "probability": <integer between 0 and 100>,
      "label": "very unlikely | unlikely | uncertain | likely | very likely",
      "reasoning": "<1-3 sentence explanation>"
    }
    """

    user_prompt = f"""
    TIME HORIZON: {horizon or "Not specified"}

    CONTEXT:
    {context}

    EVENT TO FORECAST:
    {event_to_forecast}

    Question: How likely is this event to occur within the time horizon?
    """

    try:
        res = llm.invoke(
            [SystemMessage(content=system_prompt), HumanMessage(content=user_prompt)]
        )
        raw = res.content.strip().replace("```json", "").replace("```", "")
        data = json.loads(raw)

        prob = int(data.get("probability", 50))
        prob = max(0, min(100, prob))
        label = data.get("label", "uncertain")
        reasoning = data.get("reasoning", "No reasoning provided.")
        return {"probability": prob, "label": label, "reasoning": reasoning}
    except Exception as e:
        return {
            "probability": 50,
            "label": "uncertain",
            "reasoning": f"Parsing or LLM error: {e}"
        }

run_forecast() Function Cell → Selects the scenario or uses custom inputs, passes details to the forecast engine, and formats the result.

In [ ]:
def run_forecast(selected_title, custom_context, custom_event, custom_horizon):
    scenario = scenario_map[selected_title]

    # If user chooses Custom, use their text
    if scenario["id"] == "E5":
        context = custom_context.strip()
        event_to_forecast = custom_event.strip()
        horizon = custom_horizon.strip() or "Next 6–12 months"
    else:
        context = scenario["context"]
        event_to_forecast = scenario["event_to_forecast"]
        horizon = scenario["horizon"]

    if not context or not event_to_forecast:
        return "❌ Please provide both CONTEXT and EVENT description."

    result = forecast_event(context, event_to_forecast, horizon)

    prob = result["probability"]
    label = result["label"]
    reasoning = result["reasoning"]

    md = f"""
### 🔮 Event Forecast

**Scenario:** {selected_title}
**Time Horizon:** `{horizon}`

**Event:**
> {event_to_forecast}

---

**Model Forecast:**

- **Estimated Probability:** `{prob}%`
- **Qualitative Label:** `{label}`
- **Reasoning:**
  {reasoning}

> ⚠️ This tool is for research & learning only. It is **not** financial, investment, policy, or defence advice.
"""

    return md

# Gradio

 Builds the interface where users pick a scenario, enter custom details, run predictions, and view the forecast.

In [ ]:
with gr.Blocks(title="Evaluating LLM Agents for Event Forecasting") as demo:
    gr.Markdown("## 🔮 Evaluating LLM Agents for Event Forecasting")
    gr.Markdown(
        "Pick a real-world style scenario (Samsung, Myntra, RBI, Indian Military) "
        "or define your own, and let the LLM estimate how likely the event is."
    )

    with gr.Row():
        with gr.Column():
            scenario_title = gr.Dropdown(
                choices=[s["title"] for s in SCENARIOS],
                value=SCENARIOS[0]["title"],
                label="Choose Scenario"
            )

            custom_context = gr.Textbox(
                label="Custom Context (used only for 'Custom' scenario)",
                placeholder="Example: Market trend, announcements, news, risks, signals...",
                lines=6
            )
            custom_event = gr.Textbox(
                label="Custom Event to Forecast",
                placeholder="Example: 'Myntra will double its active users this Diwali.'"
            )
            custom_horizon = gr.Textbox(
                label="Time Horizon",
                placeholder="Example: Next 3 months"
            )

            run_btn = gr.Button("Run Forecast 🚀", variant="primary")

        with gr.Column():
            output_md = gr.Markdown(label="Forecast Result")

    run_btn.click(
        fn=run_forecast,
        inputs=[scenario_title, custom_context, custom_event, custom_horizon],
        outputs=output_md
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed2de89792595be525.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
